In [13]:
!pip install pyspark


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum, desc

spark = SparkSession.builder.appName("Retail_ETL_Simulation").getOrCreate()


In [15]:
sales = spark.read.csv("sales_cleaned.csv", header=True, inferSchema=True)
inventory = spark.read.csv("inventory_cleaned.csv", header=True, inferSchema=True)


In [16]:
sales_summary = sales.groupBy("product_id").agg(
    spark_sum("quantity").alias("units_sold")
)


In [17]:
final_data = sales_summary.join(inventory, "product_id")
final_data = final_data.withColumn(
    "remaining_stock",
    final_data["stock"] - final_data["units_sold"]
)


In [18]:
final_data.write.mode("overwrite").csv("retail_etl_output", header=True)

final_data.createOrReplaceTempView("retail_table")


In [19]:
final_data.createOrReplaceTempView("retail_table")


In [20]:
top_products = spark.sql("""
    SELECT product_id, units_sold
    FROM retail_table
    ORDER BY units_sold DESC
    LIMIT 5
""")

top_products.show()


+----------+----------+
|product_id|units_sold|
+----------+----------+
|       101|         8|
|       104|         6|
|       103|         4|
|       102|         3|
+----------+----------+

